# Introduction

## Q1: Running Elastic:

To run elastic, first had to install it. Other option is to use the docker image. The installation was done using the following commands:

```bash
docker run -it \
    --rm \
    --name elasticsearch \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    docker.elastic.co/elasticsearch/elasticsearch:8.4.3
```

After running the above command, the elastic search was running on the localhost. The following command was used to check if the elastic search was running:
    
```bash
curl -X GET "localhost:9200/"
```

If you want to check the health of the elastic search, you can use the following command:

```bash
curl -X GET "localhost:9200/_cat/health?v"
```

If you want to check the indices in the elastic search, you can use the following command:

```bash
curl -X GET "localhost:9200/_cat/indices?v"
```

To check the `version.build_hash` of the elastic search, you can use the following command:

```bash
curl -X GET "localhost:9200/"
```

In [29]:
from elasticsearch import Elasticsearch
import requests
from tqdm.auto import tqdm

In [3]:
es_client = Elasticsearch('http://localhost:9200') 

In [25]:
build_hash_result = requests.get('http://localhost:9200').json()
build_hash_result.get('version').get('build_hash')  

'42f05b9372a9a4a470db3b52817899b99a76ee73'

### Getting the data:

In [26]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

# Q2: Indexing the data:

Make the course field a keyword and the rest should be text.

```
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}
```

The function that is used for adding data in elastic search is as follows:

```python

def add_documents_to_index(documents, index_name):
    es_client.indices.create(index=index_name, body=index_settings)
    for doc in documents:
        es_client.index(index=index_name, body=doc)
```

In [27]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

In [30]:
index_name = "course-questions"
es_client.indices.create(index=index_name, body=index_settings)

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

# Q3: Search:

Execute a query `"How do I execute a command in a running docker container?".`

Use only question and text fields and give question a boost of 4, and use "type": "best_fields".

Top ranking result?

In [31]:
query = {
    "query": {
        "multi_match": {
            "query": "How do I execute a command in a running docker container?",
            "fields": ["question^4", "text"],
            "type": "best_fields"
        }
    }
}

In [32]:
result = es_client.search(index=index_name, body=query)

In [33]:
result['hits']['hits'][0]['_source']

{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
 'section': '5. Deploying Machine Learning Models',
 'question': 'How do I debug a docker container?',
 'course': 'machine-learning-zoomcamp'}

In [34]:
result['hits']['hits'][0]['_score']

84.050095

In [35]:
result['hits']['hits'][0]['_source']['text']

'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)'

# Q4: Filtering:

Limit the questions to `machine-learning-zoomcamp`.
Return 3 results. What's the 3rd question returned?

* How do I debug a docker container?
* How do I copy files from a different folder into docker container’s working directory?
* How do Lambda container images work?
* How can I annotate a graph?

In [61]:
# limit the questions to machine-learning-zoomcamp
query = {
    "query": {
        "bool": {
            "filter": {
                "term": {"course": "machine-learning-zoomcamp"}
            }
        }
    },
    "size": 3
}

In [62]:
result = es_client.search(index=index_name, body=query)

In [63]:
result['hits']['hits'][2]['_source']['question']

'What if I miss a session?'

# Q5: Building a prompt:

Build a prompt to send to an LLM.

Take the records returned from Elasticsearch in Q4 and use this template to build the context. Separate context entries by two linebreaks `(\n\n)`

```
context_template = """
Q: {question}
A: {text}
""".strip()
```

use the context you just created along with the "How do I execute a command in a running docker container?" question to construct a prompt using the template below:

```
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
```

What's the length of the resulting prompt? (use the len function)


In [64]:
context_template = """
Q: {question}
A: {text}
""".strip()

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION. 

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [65]:
context = "\n\n".join([context_template.format(**hit['_source']) for hit in result['hits']['hits']])

In [69]:
question = "What if I miss a session?"

In [70]:
prompt = prompt_template.format(question=question, context=context)

In [71]:
len(prompt)

1244

# Q6: Tokens:

When we use the OpenAI Platform, we're charged by the number of tokens we send in our prompt and receive in the response.

The OpenAI python package uses `tiktoken` for tokenization:

the number of tokens in our query:
```
encoding = tiktoken.encoding_for_model("gpt-4o")
```

Use the `encode` function. How many tokens does our prompt have?


In [72]:
# How many tokens does our prompt have?
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")

tokens = encoding.encode(prompt)

len(tokens)

275

In [73]:
encoding.decode_single_token_bytes(63842)

b"You're"